# Feature Engineering - Amazon Sales Data

In this notebook, we will create new features from the cleaned dataset.  
The goal of feature engineering is to make the dataset more informative for machine learning models.

## Step 0: Load Processed Data
We load the pre-cleaned dataset (`amazon_sales_processed.csv`) stored in the `data/` folder.


In [3]:
import pandas as pd

df = pd.read_csv("../data/processed/amazon_sales_processed.csv")
df.head()



,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,...,actual_price_num,discounted_price_num,discount_pct_num,rating_num,rating_count_num,category_main,discount_pct_calc,discount_pct_diff,discount_amount,popularity_score
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...",...,1099.0,399.0,64.0,4.2,24269.0,Computers&Accessories,63.694268,0.305732,700.0,42.407384
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...",...,349.0,199.0,43.0,4.0,43994.0,Computers&Accessories,42.979943,0.020057,150.0,42.767325
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...",...,1899.0,199.0,90.0,3.9,7928.0,Computers&Accessories,89.520800,0.479200,1700.0,35.015301
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...",...,699.0,329.0,53.0,4.2,94363.0,Computers&Accessories,52.932761,0.067239,370.0,48.110643
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...",...,399.0,154.0,61.0,4.2,16905.0,Computers&Accessories,61.403509,0.403509,245.0,40.888780


## Step 2.2: Inspect Data Types & Categorize Columns

Before performing feature engineering, we must understand which columns are:
- **Numerical** (e.g., sales, price, quantity, revenue)
- **Categorical** (e.g., category, region, product_id, customer_segment)
- **Date/Time** (e.g., order_date, ship_date)

This categorization helps us decide the right transformations for each type.


In [4]:
# Check column data types
df.info()

# Quick look at categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical Columns:", categorical_cols)

# Quick look at numerical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Numerical Columns:", numerical_cols)

# Quick look at datetime columns (if already parsed)
datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()
print("Datetime Columns:", datetime_cols)

# If date columns are still 'object', we can check like this:
potential_dates = [col for col in df.columns if 'date' in col.lower()]
print("Potential Date Columns:", potential_dates)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_id            1465 non-null   object 
 1   product_name          1465 non-null   object 
 2   category              1465 non-null   object 
 3   discounted_price      1465 non-null   object 
 4   actual_price          1465 non-null   object 
 5   discount_percentage   1465 non-null   object 
 6   rating                1465 non-null   object 
 7   rating_count          1463 non-null   object 
 8   about_product         1465 non-null   object 
 9   user_id               1465 non-null   object 
 10  user_name             1465 non-null   object 
 11  review_id             1465 non-null   object 
 12  review_title          1465 non-null   object 
 13  review_content        1465 non-null   object 
 14  img_link              1465 non-null   object 
 15  product_link         

In [5]:
# Check missing values
df.isnull().sum()

# Fill missing ratings with median
df["rating_num"].fillna(df["rating_num"].median(), inplace=True)

# Fill missing rating_count with median
df["rating_count_num"].fillna(df["rating_count_num"].median(), inplace=True)


C:\Users\91969\AppData\Local\Temp\ipykernel_71720\3972743001.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["rating_num"].fillna(df["rating_num"].median(), inplace=True)
C:\Users\91969\AppData\Local\Temp\ipykernel_71720\3972743001.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [6]:
df["price_diff"] = df["actual_price_num"] - df["discounted_price_num"]
df["discount_ratio"] = df["discount_amount"] / df["actual_price_num"]


In [7]:
import numpy as np

df["weighted_rating"] = df["rating_num"] * np.log1p(df["rating_count_num"])
df["high_rating"] = (df["rating_num"] >= 4).astype(int)


In [8]:
from sklearn.preprocessing import LabelEncoder

# Label Encode product_id
le = LabelEncoder()
df["product_id_encoded"] = le.fit_transform(df["product_id"])

# One-hot encode category_main
df = pd.get_dummies(df, columns=["category_main"], drop_first=True)


In [9]:
df["review_length"] = df["review_content"].apply(lambda x: len(str(x).split()))
df["about_length"] = df["about_product"].apply(lambda x: len(str(x).split()))


In [10]:
output_path = "../data/processed/amazon_sales_features.csv"
df.to_csv(output_path, index=False)
print("✅ Feature-engineered dataset saved:", output_path)


✅ Feature-engineered dataset saved: ../data/processed/amazon_sales_features.csv
